# Point cloud registration with Easy Open3D

In [1]:
from easy_o3d import utils
from easy_o3d.registration import IterativeClosestPoint, FastGlobalRegistration, RANSAC, ICPTypes, KernelTypes
from easy_o3d import set_logger_level

import logging
import copy
import open3d as o3d
import numpy as np
import json

set_logger_level(logging.DEBUG)

## Load and visualize point clouds

In [2]:
# Load ground truth pose
with open("tests/test_data/ground_truth_pose.json") as f:
    gt_pose_data = json.load(f)
rotation = gt_pose_data["rotation_quaternion"]
translation = gt_pose_data["translation_xyz"]
gt_pose = utils.get_transformation_matrix_from_quaternion(rotation_wxyz=rotation,
                                                          translation_xyz=translation)

source_path = "tests/test_data/suzanne.ply"
source = utils.eval_data(data=source_path, number_of_points=10000)

source = utils.process_point_cloud(point_cloud=source,
                                   downsample=utils.DownsampleTypes.VOXEL,
                                   downsample_factor=0.01)

source = utils.process_point_cloud(point_cloud=source,
                                   estimate_normals=True,
                                   search_param_knn=30,
                                   search_param_radius=0.02,
                                   recalculate_normals=True)

_, source_feature = utils.process_point_cloud(point_cloud=source,
                                              compute_feature=True,
                                              search_param_knn=100,
                                              search_param_radius=0.05)

target_path = "tests/test_data/suzanne_on_chair.ply"
target = utils.eval_data(data=target_path, number_of_points=100000)

target = utils.process_point_cloud(point_cloud=target,
                                   downsample=utils.DownsampleTypes.VOXEL,
                                   downsample_factor=0.01)

target = utils.process_point_cloud(point_cloud=target,
                                   estimate_normals=True,
                                   search_param_knn=30,
                                   search_param_radius=0.02,
                                   recalculate_normals=True)

_, target_feature = utils.process_point_cloud(point_cloud=target,
                                              compute_feature=True,
                                              search_param_knn=100,
                                              search_param_radius=0.05)

utils.draw_geometries(geometries=[source, target])

DEBUG - 2021-04-23 14:40:32,420 - utils - Trying to convert mesh to point cloud.
DEBUG - 2021-04-23 14:40:32,425 - utils - DownsampleTypes.VOXEL downsampling point cloud  with factor 0.01.
DEBUG - 2021-04-23 14:40:32,426 - utils - Number of points before downsampling: 10000
DEBUG - 2021-04-23 14:40:32,426 - utils - Number of points after downsampling: 1371
DEBUG - 2021-04-23 14:40:32,427 - utils - Processing took 0.0017361640930175781 seconds.
DEBUG - 2021-04-23 14:40:32,427 - utils - Estimating point cloud normals using method SearchParamTypes.HYBRID.
DEBUG - 2021-04-23 14:40:32,430 - utils - Processing took 0.0031120777130126953 seconds.
DEBUG - 2021-04-23 14:40:32,431 - utils - Computing FPFH features using method SearchParamTypes.HYBRID.
DEBUG - 2021-04-23 14:40:32,436 - utils - Processing took 0.005790233612060547 seconds.
DEBUG - 2021-04-23 14:40:32,437 - utils - Trying to convert mesh to point cloud.
DEBUG - 2021-04-23 14:40:32,459 - utils - DownsampleTypes.VOXEL downsampling po

## RANSAC

In [3]:
ransac = RANSAC()
ransac_result = ransac.run(source=source,
                           target=target,
                           source_feature=source_feature,
                           target_feature=target_feature,
                           draw=True,
                           overwrite_colors=True)

print(np.linalg.norm(gt_pose - ransac_result.transformation))

DEBUG - 2021-04-23 14:40:34,202 - interfaces - Couldn't find data PointCloud with 1371 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:40:34,203 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:40:34,203 - interfaces - Couldn't find data PointCloud with 31685 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:40:34,203 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:40:34,280 - registration - RANSAC took 0.07818365097045898 seconds.
DEBUG - 2021-04-23 14:40:34,281 - registration - RANSAC result: fitness=0.3952941176470588, inlier_rmse=0.008376283022820095.
DEBUG - 2021-04-23 14:40:34,281 - interfaces - Couldn't find data PointCloud with 1371 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:40:34,282 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:40:34,282 - interfaces - Couldn't find data PointCloud with 31685 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:40:34,282 - utils - Data is point cloud. Returning.


0.07246334915879676


## Fast Global Registration

In [4]:
fgr = FastGlobalRegistration()
fgr_result = fgr.run(source=source,
                     target=target,
                     source_feature=source_feature,
                     target_feature=target_feature,
                     draw=True,
                     overwrite_colors=True)

print(np.linalg.norm(gt_pose - fgr_result.transformation))

DEBUG - 2021-04-23 14:40:47,294 - interfaces - Couldn't find data PointCloud with 1371 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:40:47,295 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:40:47,295 - interfaces - Couldn't find data PointCloud with 31685 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:40:47,295 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:40:47,420 - registration - FGR took 0.1257643699645996 seconds.
DEBUG - 2021-04-23 14:40:47,421 - registration - FGR result: fitness=0.700948212983224, inlier_rmse=0.0035745115522943763.
DEBUG - 2021-04-23 14:40:47,421 - interfaces - Couldn't find data PointCloud with 1371 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:40:47,422 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:40:47,422 - interfaces - Couldn't find data PointCloud with 31685 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:40:47,422 - utils - Data is point cloud. Returning.


0.06423980513009363


## Iterative Closest Point

In [5]:
icp = IterativeClosestPoint(estimation_method=ICPTypes.PLANE)
result = icp.run(source=source,
                 target=target,
                 init=ransac_result.transformation,
                 max_iteration=100,
                 draw=True,
                 overwrite_colors=True)

print(np.linalg.norm(gt_pose - result.transformation))

DEBUG - 2021-04-23 14:41:12,225 - interfaces - Couldn't find data PointCloud with 1371 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:41:12,225 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:41:12,226 - interfaces - Couldn't find data PointCloud with 31685 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:41:12,226 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:41:12,294 - registration - ICP took 0.06990551948547363 seconds.
DEBUG - 2021-04-23 14:41:12,295 - registration - ICP result: fitness=0.5178701677607586, inlier_rmse=0.0028388773817843807.
DEBUG - 2021-04-23 14:41:12,295 - interfaces - Couldn't find data PointCloud with 1371 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:41:12,296 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:41:12,296 - interfaces - Couldn't find data PointCloud with 31685 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:41:12,296 - utils - Data is point cloud. Returning.


0.0036205802498571675


## Load camera parameters from BlenderProc BopWriter

In [6]:
scene_id = 2
path_to_scene_camera_json = "tests/test_data/bop_data/obj_of_interest/train_pbr/000000/scene_camera.json"
path_to_camera_json = "tests/test_data/bop_data/obj_of_interest/camera.json"
output_path = "tests/test_data"

camera_parameters = utils.get_camera_parameters_from_blenderproc_bopwriter(scene_id,
                                                                           path_to_scene_camera_json,
                                                                           path_to_camera_json,
                                                                           output_path)

## Load RGB-D image

In [7]:
camera_parameters = o3d.io.read_pinhole_camera_parameters("tests/test_data/camera_parameters.json")
color = f"tests/test_data/bop_data/obj_of_interest/train_pbr/000000/rgb/000002.png"
depth = f"tests/test_data/bop_data/obj_of_interest/train_pbr/000000/depth/000002.png"

target_rgbd = utils.convert_rgbd_image_to_point_cloud(rgbd_image_or_path=[color, depth],
                                                      camera_intrinsic=camera_parameters.intrinsic,
                                                      camera_extrinsic=camera_parameters.extrinsic,
                                                      depth_scale=1000.0,
                                                      depth_trunc=2.0)

target_rgbd = utils.process_point_cloud(point_cloud=target_rgbd,
                                        downsample=utils.DownsampleTypes.VOXEL,
                                        downsample_factor=0.01,
                                        remove_outlier=utils.OutlierTypes.RADIUS,
                                        outlier_std_ratio=1.0,
                                        search_param_knn=20,
                                        search_param_radius=0.2)

target_rgbd = utils.process_point_cloud(point_cloud=target_rgbd,
                                        estimate_normals=True,
                                        search_param_knn=30,
                                        search_param_radius=0.02)

_, target_rgbd_feature = utils.process_point_cloud(point_cloud=target_rgbd,
                                                   compute_feature=True,
                                                   search_param_knn=100,
                                                   search_param_radius=0.05)

utils.draw_geometries(geometries=[target_rgbd])

DEBUG - 2021-04-23 14:41:27,420 - utils - DownsampleTypes.VOXEL downsampling point cloud  with factor 0.01.
DEBUG - 2021-04-23 14:41:27,420 - utils - Number of points before downsampling: 59296
DEBUG - 2021-04-23 14:41:27,424 - utils - Number of points after downsampling: 12197
DEBUG - 2021-04-23 14:41:27,517 - utils - Removed 95 outliers.
DEBUG - 2021-04-23 14:41:27,518 - utils - Processing took 0.09809350967407227 seconds.
DEBUG - 2021-04-23 14:41:27,519 - utils - Estimating point cloud normals using method SearchParamTypes.HYBRID.
DEBUG - 2021-04-23 14:41:27,522 - utils - Processing took 0.0038962364196777344 seconds.
DEBUG - 2021-04-23 14:41:27,523 - utils - Computing FPFH features using method SearchParamTypes.HYBRID.
DEBUG - 2021-04-23 14:41:27,565 - utils - Processing took 0.04212665557861328 seconds.


## RANSAC

In [8]:
ransac = RANSAC()
ransac_result = ransac.run(source=source,
                           target=target_rgbd,
                           source_feature=source_feature,
                           target_feature=target_rgbd_feature,
                           draw=True,
                           overwrite_colors=True)

print(np.linalg.norm(gt_pose - ransac_result.transformation))

DEBUG - 2021-04-23 14:41:29,360 - interfaces - Couldn't find data PointCloud with 1371 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:41:29,361 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:41:29,361 - interfaces - Couldn't find data PointCloud with 12102 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:41:29,361 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:41:29,410 - registration - RANSAC took 0.05005502700805664 seconds.
DEBUG - 2021-04-23 14:41:29,412 - registration - RANSAC result: fitness=0.18996415770609318, inlier_rmse=0.00904222400015424.
DEBUG - 2021-04-23 14:41:29,412 - interfaces - Couldn't find data PointCloud with 1371 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:41:29,415 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:41:29,416 - interfaces - Couldn't find data PointCloud with 12102 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:41:29,417 - utils - Data is point cloud. Returning.


0.1884329898661514


## Fast Global Registration

In [9]:
fgr = FastGlobalRegistration()
fgr_result = fgr.run(source=source,
                     target=target_rgbd,
                     source_feature=source_feature,
                     target_feature=target_rgbd_feature,
                     draw=True,
                     overwrite_colors=True)

print(np.linalg.norm(gt_pose - fgr_result.transformation))

DEBUG - 2021-04-23 14:41:30,586 - interfaces - Couldn't find data PointCloud with 1371 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:41:30,586 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:41:30,586 - interfaces - Couldn't find data PointCloud with 12102 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:41:30,587 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:41:30,634 - registration - FGR took 0.048516035079956055 seconds.
DEBUG - 2021-04-23 14:41:30,635 - registration - FGR result: fitness=0.18161925601750548, inlier_rmse=0.003639157060406545.
DEBUG - 2021-04-23 14:41:30,636 - interfaces - Couldn't find data PointCloud with 1371 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:41:30,636 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:41:30,636 - interfaces - Couldn't find data PointCloud with 12102 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:41:30,637 - utils - Data is point cloud. Returning.


0.14847635650156613


## Iterative Closest Point

In [10]:
icp = IterativeClosestPoint()
result = icp.run(source=source,
                 target=target_rgbd,
                 estimation_method=ICPTypes.PLANE,
                 init=ransac_result.transformation,
                 crop_target_around_source=True,
                 crop_scale=1.5,
                 max_iteration=100,
                 max_correspondence_distance=0.004,
                 draw=True,
                 overwrite_colors=False)

print(np.linalg.norm(gt_pose - result.transformation))

DEBUG - 2021-04-23 14:41:31,074 - interfaces - Couldn't find data PointCloud with 1371 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:41:31,075 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:41:31,075 - interfaces - Couldn't find data PointCloud with 12102 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:41:31,075 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:41:31,143 - registration - ICP took 0.06880807876586914 seconds.
DEBUG - 2021-04-23 14:41:31,146 - registration - ICP result: fitness=0.274981765134938, inlier_rmse=0.0028377138444532495.
DEBUG - 2021-04-23 14:41:31,147 - interfaces - Couldn't find data PointCloud with 1371 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:41:31,148 - utils - Data is point cloud. Returning.
DEBUG - 2021-04-23 14:41:31,148 - interfaces - Couldn't find data PointCloud with 1788 points. in cache. Re-evaluating.
DEBUG - 2021-04-23 14:41:31,149 - utils - Data is point cloud. Returning.


0.0038702285185791886
